In [ ]:
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.cloud import storage
from google.oauth2 import service_account


def authenticate(credentials):
    """Authenticate using the provided credentials."""
    return service_account.Credentials.from_service_account_info(credentials)


def download_file(drive_service, file_id):
    """Download a file from Google Drive."""
    request = drive_service.files().get_media(fileId=file_id)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    file_stream.seek(0)  # Reset the stream position to the beginning
    return file_stream


def upload_file(client, bucket_name, file_stream, destination_blob_name):
    """Upload a file to Google Cloud Storage."""
    blob = client.bucket(bucket_name).blob(destination_blob_name)
    file_stream.seek(0)  # Reset the stream position to the beginning
    blob.upload_from_file(file_stream, content_type='application/octet-stream')


def transfer_files(credentials, project_id, bucket_name, file_urls, file_names, folder_path):
    """Transfer files from Google Drive to Google Cloud Storage."""
    # Authenticate using the provided credentials
    drive_service = build('drive', 'v3', credentials=credentials)

    # Create a client for Google Cloud Storage directly using the credentials
    client = storage.Client(credentials=credentials, project=project_id)

    # Iterate over the file URLs and names simultaneously
    for file_url, file_name in zip(file_urls, file_names):
        # Extract the file ID from the URL
        file_id = file_url.split('/')[-1]

        # Download the file from Google Drive
        file_stream = download_file(drive_service, file_id)

        # Upload the file to Google Cloud Storage with the desired folder path
        destination_blob_name = folder_path + file_name
        upload_file(client, bucket_name, file_stream, destination_blob_name)

        print('File transfer completed for file ID:', file_id)

    print('All files transferred.')


# Define the credentials (create service with access to Storage and Drive APIs, and create a key for it in order to receive this data)
credentials = {
  "type": "service_account",
  "project_id": "cedar-pottery-388916",
  "private_key_id": "",
  "private_key": "",
  "client_email": "",
  "client_id": "",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/drive-895%40cedar-pottery-388916.iam.gserviceaccount.com",
  "universe_domain": "googleapis.com"
}

# Define the project ID
project_id = 'cedar-pottery-388916'

# Define the destination bucket name in Google Cloud Storage
bucket_name = 'bucket123321'

# Define a list of file URLs
file_urls = [
    'https://drive.google.com/file/d/1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu',
    # 'https://drive.google.com/file/d/1mwNNdOMSNty6WumYdH9FJNJZJYQ6oD1c',
    # 'https://drive.google.com/file/d/1TT4ARRIV6i2fO1b5yb0aSUkjhxMb9u6g'
]

# Define the corresponding file names
file_names = ['business.pkl', 'review.json', 'user.parquet']

# Define the destination folder path within the bucket
folder_path = 'yelp/'

# Authenticate using the provided credentials
credentials = authenticate(credentials)

# Call the transfer_files function to execute the file transfer
transfer_files(credentials, project_id, bucket_name, file_urls, file_names, folder_path)
